# Introduction to python decorators
	An introduction to the different levels
	of how to use a decorator function
	By: Paal Pedersen
	Email: paal.pedersen@geodata.no


## Lets explore a faulty decorator

In [164]:
def faulty(func):
    ''' a first level decorator, which get evaluated at decorating level '''
    print('This runs before the function')
    return func

 ### Note: When we created our faulty decorator function
  we can see that this is just a function that
  accepts one argument/variable it prints one
  line before it returns the variable

In [165]:
@faulty # top level get evaluated here
def function1():
    print('Inside function1')
    print()

This runs before the function


 ### On top of the second level we see this @ before faulty decorater. 
 this is the syntax for creating a wrapper around
  the function. but our decorator is missing a wrapper.
  So we can see the output from the decorator immediately.

In [3]:
# lets call our decorated function
function1()

Inside function1



### Since there were no wrapper, it still works like there is no decorator on top

## Lets explore a working decorator

In [4]:
def decorator1(func):
    # Every thing we do here will be evaluated at decorator instance
    print('Remember!')
    # So we need a wrapper:
    def argument_wrapper(*args):
        print('Inside the second level')
        print('Lets take a look at the arguments')
        print(*args)
        print('or as a tuple')
        print(args)
        print()
        return func(*args)
    
    # we must return the wrapper
    return argument_wrapper

In [5]:
@decorator1
def function2(*args):
    print('Inside function2')
    print(f'We got {len(args)} argument(s)')

Remember!


### Notice the first print output!


In [6]:
function2(1, 2, 3)

Inside the second level
Lets take a look at the arguments
1 2 3
or as a tuple
(1, 2, 3)

Inside function2
We got 3 argument(s)


__Notice how we can se both functions in action__

### What do you think will happen if we decorate function2 with the faulty decorator?

In [7]:
from difflib import SequenceMatcher # for correcting input typos
answer = input('Your answer:\n').title()
correct = ''.join(map(chr, (84, 104, 105, 115, 32, 114, 117, 110, 115, 32, 98, 101, 102, 111, 114, 101, 32, 116, 104, 101, 32, 102, 117, 110, 99, 116, 105, 111, 110)))
if SequenceMatcher(None, answer, correct).ratio() > 0.8:
    print(f'Your answer: "{correct}" is correct!')
    print()
else:
    print('Correct answer was:')

Your answer:
DSGSDgsDG
Correct answer was:


In [8]:
@faulty
def function3(*args):
    print('Inside function3')
    print(f'We got {len(args)} argument(s)')

This runs before the function


### And then we can notice the print when calling the function

In [9]:
function3(1,2,3)

Inside function3
We got 3 argument(s)


## Lets dig deeper

In [155]:
def decorator2(input_=()):
    def function_wrapper(func):
        def argument_wrapper(*args):
            print(f'Decorator arguments: {input_}')
            print(f'Got a function named: {func.__name__}')
            print(f'with arguments: {args}')
            print('Complete function syntax:')
            print(f'{func.__name__}{args}')
            result = func(*args)
            print('Inside the decorator again')
            print(f'got result: {result}')
            print('Should we change the result?')
            print('square every value')
            result = tuple(i ** 2 for i in result)
            print(f'new result: {result}')
            return result
        return argument_wrapper
    return function_wrapper

### Notice the one significant different here, and that is the arguments for decorator

In [156]:
@decorator2(input_=('input to the decorator', 1, 2, 'this', 'is', 'fun'))
def function4(*args):
    print('Inside function4 with arguments')
    return args

In [157]:
res = function4(1,2,3)

Decorator arguments: ('input to the decorator', 1, 2, 'this', 'is', 'fun')
Got a function named: function4
with arguments: (1, 2, 3)
Complete function syntax:
function4(1, 2, 3)
Inside function4 with arguments
Inside the decorator again
got result: (1, 2, 3)
Should we change the result?
square every value
new result: (1, 4, 9)


In [158]:
print(res)

(1, 4, 9)


### Lets do something usefull with decorator arguments

In [159]:
def typecheckArguments(_types=()):
    def check_types(*args):
        ''' Helper function '''
        assert len(args) == len(_types)
        for index, (_type, arg) in enumerate(zip(_types, args)):
            if not isinstance(arg, _type):
                print(f'argument at index {index+1} with value: {arg} not of type: {_type}')
                
    def funcwrapper(func):
        def argwrapper(*args):
            check_types(*args)
            return func(*args)

        return argwrapper
    return funcwrapper

In [160]:
@typecheckArguments(_types=(int, int))
def takes_two_ints(a, b):
    return a + b

In [161]:
print('result', takes_two_ints(1, 3))

result 4


Calling takes_two_ints with one int, and one float

In [162]:
print('result', takes_two_ints(1, 1.1))

argument at index 2 with value: 1.1 not of type: <class 'int'>
result 2.1


 This is more usefull, it gives us a nice feedback in the console when we enter a wrong type. 
But then it continues with out raising any errors, and that is because you can add ints and floats.
But what if we enter a string?

In [163]:
print('result', takes_two_ints(1, '3'))

argument at index 2 with value: 3 not of type: <class 'int'>


TypeError: unsupported operand type(s) for +: 'int' and 'str'

Here we notice our typechecker print the fault before we encounter an error

## Lets create another typechecker

In [7]:
def typecheckArguments2(allowed_types=()):
    def check_types(*args):
        filtered_args = []
        for arg in args:
            if type(arg) not in allowed_types:
                print(f'Discarding invalid type: {type(arg)}, value: {arg}')
            else:
                filtered_args.append(arg)
        return filtered_args

    def wrapper(func):
        def _wrapper(*args):
            # We call the check_types here
            filtered = check_types(*args)
            print(f'using flitered values: {tuple(filtered)}')
            res = func(*filtered)
            print(f'result: {res}')
            return res
        return _wrapper
    return wrapper

In [8]:
@typecheckArguments2(allowed_types=(int, float))
def acceptsNumbers(*args):
    print('summing...')
    return sum(args)

In [9]:
acceptsNumbers(1, 2, 3, 5.5, int, 'str', str)

Discarding invalid type: <class 'type'>, value: <class 'int'>
Discarding invalid type: <class 'str'>, value: str
Discarding invalid type: <class 'type'>, value: <class 'str'>
using flitered values: (1, 2, 3, 5.5)
summing...
result: 11.5


11.5

Here we filtered out argmuments of wrong type in a greedy manner

## Lets extend some functionality

In [10]:
def typecheckArguments3(allowed_types=(), cast=True, cast_to=float):
    
    def cast_filter(value_of_type):
        try:
            cast_to(value_of_type)
            return True
        except:
            print(f'Discarding invalid type: {type(value_of_type)}, value: {value_of_type}')
            return False

    def check_types(*args):
        clean_args = []
        correct_types = []
        for arg in args:
            if type(arg) not in allowed_types:
                correct_types.append(arg)
            else:
                clean_args.append(arg)

        if cast:
            # if cast is True, we filter out bad casts
            correct_types = list(map(cast_to, filter(cast_filter, correct_types)))
            clean_args.extend(correct_types)
            
        return tuple(clean_args)

    def wrapper(func):
        def _wrapper(*args):
            clean = check_types(*args)
            res = func(*clean)
            print('result:', res)
            return res

        return _wrapper

    return wrapper

In [11]:
@typecheckArguments3(allowed_types=(int, float))
def acceptsNumbersAstheyAre(*args):
    print('summing...')
    return sum(args)

In [12]:
acceptsNumbersAstheyAre(1, 2, 3, '5', 's', None)

Discarding invalid type: <class 'str'>, value: s
Discarding invalid type: <class 'NoneType'>, value: None
summing...
result: 11.0


11.0

In [13]:
@typecheckArguments3(allowed_types=(str,), cast_to=str)
def acceptsAny(*args):
    print('joining...')
    return ''.join(args)

In [14]:
acceptsAny(1,23, 'hello World', 321)

joining...
result: hello World123321


'hello World123321'

 Notice! how our logic got twisted

## Lets fix it
### Notice the different inputs to the decorator afterwards

In [166]:
def typecheckArguments4(allowed_types=(), cast=True, cast_to=str):
    
    def cast_filter(value_of_type):
        try:
            cast_to(value_of_type)
            return True
        except:
            print(f'Discarding invalid type: {type(value_of_type)}, value: {value_of_type}')
            return False
        
    def check_type_filter(arg):
        keep = type(arg) in allowed_types
        if not keep:
            print(f'Discarding invalid type: {type(arg)}, value: {arg}')
        return keep

    def check_types(*args):
        if cast:
            # if cast is True, we filter out bad casts
            clean = tuple(map(cast_to, filter(cast_filter, args)))
            return clean
        else:
            clean = tuple(filter(check_type_filter, args))
            return clean

    def wrapper(func):
        def _wrapper(*args):
            clean = check_types(*args)
            res = func(*clean)
            print('result:', res)
            return res
        
        return _wrapper
    return wrapper

In [167]:
@typecheckArguments4(allowed_types=(str,), cast=False)
def acceptsString(*args):
    print('joining...')
    return ''.join(args)

In [168]:
acceptsString(1,2,3 ,'Hello', 4,5,6)

Discarding invalid type: <class 'int'>, value: 1
Discarding invalid type: <class 'int'>, value: 2
Discarding invalid type: <class 'int'>, value: 3
Discarding invalid type: <class 'int'>, value: 4
Discarding invalid type: <class 'int'>, value: 5
Discarding invalid type: <class 'int'>, value: 6
joining...
result: Hello


'Hello'

In [169]:
@typecheckArguments4(cast_to=str)
def acceptsAny2(*args):
    print('joining...')
    return ''.join(args)

In [170]:
acceptsAny2(1,2,3 ,'Hello', 4,5,6, int)

joining...
result: 123Hello456<class 'int'>


"123Hello456<class 'int'>"

### Notice how much cleaner our code became when we dropped the excessive use of list declarations, we also improved our logic drastically.

## Next up class decorators!

In [15]:
class SimpleDecorator(object):
    def __init__(self, func):
        self.func = func
    
    def __call__(self, *args):
        
        print('call beeing made')
        print('using args:', args)
        result = self.func(*args)
        print('result:', result)
        return result


The class takes one input, func. This is the decorated function<br />
then we implement the Data Model method __call__.<br />
We could initiate a new object from our class:<br />
s = SimpleDecorator(some_earlier_defined_function)<br />

Then this is when __call__ gets called:<br />
s() # when we add the parenteses.<br />

In [16]:
@SimpleDecorator
def simple(*args):
    print('inside simple with args:', *args)
    return args


In [17]:
simple(1, 2, 3)

call beeing made
using args: (1, 2, 3)
inside simple with args: 1 2 3
result: (1, 2, 3)


(1, 2, 3)

A we can see its quite simple to create a class decorator. <br />
It would also make our decorators cleaner to study. <br />


In [18]:
print(simple)

If we print our function, we can see that it belogns to SimpleDecorator<br />
So simple is now a method of SimpleDecorator.<br />

But this look ugly when we print. Could we make it better? <br />

In [171]:
class DecoratorWithArgs(object):
    def __init__(self, *args):
        self.args = args

    def __call__(self, func):
        print(f'Inside __call__')
        def function_wrapper(*args, **kwargs):
            print("'Inside function_wrapper()")
            print("Decorator arguments:", self.args)
            result = func(*args, **kwargs)
            print(f'Result: {result}')
        return function_wrapper

@DecoratorWithArgs('arg1', 2, 3,4)
def simple_func(*args, **kwargs):
    kwargs['key']='unlocked'
    return args, kwargs


Inside __call__


Remember! When we build a new wrapper under __call__ everything a the top level be evaluated a decorating instance. 

In [172]:
simple_func(9,8,7)

'Inside function_wrapper()
Decorator arguments: ('arg1', 2, 3, 4)
Result: ((9, 8, 7), {'key': 'unlocked'})


Our wrapper seams to work as expected. <br/>
### Lets try something weird

In [70]:
class NonPythonicDecorator(object):
    def __init__(self, func):
        self.func = func
    
    def setArguments(self, *args):
        self.args = args
    
    def setKeyvaluearguments(self, **kwargs):
        self.kwargs = kwargs
        
    def __call__(self,  *args, **kwargs):
        print(f'function args: {args}')
        print(f'function kwargs: {kwargs}')
        print(f'result : {self.func(*args, **kwargs)}')
        print(f'self.args: {self.args}')
        print(f'self.kwargs: {self.kwargs}')
        print(f'swooped args and kwargs result: {self.func(*self.args, **self.kwargs)}')
        combined_args = (*args , *self.args)
        combined_kwargs = {**kwargs, **self.kwargs}
        print(f'combined result: {self.func(*combined_args, **combined_kwargs)}')
        

s = NonPythonicDecorator # We leve blank to avoid init beeing called
s.setArguments(s, 7,8,9) # we call methods, and send in s as self
s.setKeyvaluearguments(s, l=2) # we call som other method where we send in s as self.

@s # We can now decorate with s, which will take next vaiable function name to initialise self, and __call__ will wrapp the args
def simple_func(*args, **kwargs):
    return args, kwargs

simple_func(1,2,3, k=5)

function args: (1, 2, 3)
function kwargs: {'k': 5}
result : ((1, 2, 3), {'k': 5})
self.args: (7, 8, 9)
self.kwargs: {'l': 2}
swooped args and kwargs result: ((7, 8, 9), {'l': 2})
combined result: ((1, 2, 3, 7, 8, 9), {'k': 5, 'l': 2})


That was weird, but it works as expcetd. but the syntaxt to create s and to place s in the methods are ugly, and should be avoided I think.

## Lets explorer a couple of builtin decorators

In [173]:
class UsingClassMethods(object):
    
    # slots define allowed class attributes
    __slots__ = ['func']
    
    def __init__(self, func):
        self.func = func
        # we want the function to be set to the instance of the class
        # in order to get its original name
        self.setattribute('func', func)
    
    # now the input function is set as an attribute of the class instance
    @classmethod
    def setattribute(cls, name, value):
        setattr(cls, name, value)
    
    # call will behave differently by applying its class instance
    def __call__(self, *args):
        print('call beeing made')
        print('using args:', args)
        result = self.func(*args)
        result = result[1:] # remove cls from result
        print('result:', result)
        return result
    
    # our str must reach into class in order to find the function
    @classmethod
    def __str__(cls):
        func = getattr(cls, 'func')
        # since func is set to the class we can extract its original name
        return f'<class {cls.__name__}>.{func.__name__}'
    
    # repr is the method called to the console output when no print is given
    @classmethod
    def __repr__(cls):
        func = getattr(cls, 'func')
        return f'<class {cls.__name__}>.{func.__name__}'

In [174]:
@UsingClassMethods
def its_simple(*args):
    print('inside simple with args:', *args)
    return args

In [175]:
its_simple # testing __repr__ method

<class UsingClassMethods>.its_simple

In [176]:
its_simple(1,2,3)

call beeing made
using args: (1, 2, 3)
inside simple with args: <class UsingClassMethods>.its_simple 1 2 3
result: (1, 2, 3)


(1, 2, 3)

When we look inside its_simple we can see that the class is beeing <br />
sent together with the args. </br>
We manage to represent the class in our own way, but we also destroyed the <br />
once simple class. </br>
We also introduce a new decorator @classmethod. which we can not find in our code, <br />
and that is because its one of the builtin class decorators. <br />
<br />
## Lets dig deeper
We also have @staticmethod, @property ....<br />
@property is to mimic a privat method<br />
@staticmethod is a method of a class, which does not<br />
accepts self. and it can be called directyl without initiatig the class <br />



In [76]:
class ImplementsStatic(object):
    
    # No init
    
    @staticmethod
    def staticfunction(*args):
        print(args)

In [77]:
ImplementsStatic.staticfunction(1,2,3)

(1, 2, 3)


__A static method can be called directly on the class instance without ever initializing the class!__ <br/>

## Lets look at the @property decorator

In [132]:
class ImplementsProperty(object):
    def __init__(self, name, secret):
        self.__name__ = name
        self._secret = secret
        
        
    @property
    def name(self):
        print('accessing name property')
        return self.__name__
    
    @name.setter
    def name(self, name):
        print('Changing name')
        self.__dict__['__name__'] = name
    
    @property
    def secret(self):
        if hasattr(self, '__name__'):
            if getattr(self, '__name__') == 'The Secret Name':
                print(f'"{self.__name__}" Accessing secret')
                return self._secret
            else:
                print('Invalid access')
        else:
            print('Invalid access')
    
    def accessor(self):
        
        print(f'the secret is: {self.secret}')
        
    @secret.setter
    def secret(self, sec):
        if hasattr(self, '__name__'):
            if getattr(self, '__name__') == 'The Secret Name':
                print(f'"{self.__name__}" Changing secret')
                self._secret = sec
            else:
                print('Invalid access')
        else:
            print('Invalid access')
    

In [133]:
I = ImplementsProperty('my secret', 'mee')
I.name = 'new name'

Changing name


In [134]:
I.name

accessing name property


'new name'

In [135]:
I.secret

Invalid access


In [136]:
I.__name__ = 'The Secret Name'

In [137]:
I.accessor()

"The Secret Name" Accessing secret
the secret is: mee


In [138]:
I.secret = 'New secret1'

"The Secret Name" Changing secret


In [139]:
I._secret = 'New secret2' # go straith the source attribute

In [140]:
I.name = 'another name'

Changing name


In [148]:
I._secret

'New secret2'

Now we got a sence of what the @property can do. It creates wrappers for how to access attributes on a class. 

These things are quite confusing, S I advice to play around ass much as possible to get the deeper knowedge of what is going on.